## Pipeline

[tensorflow-object-detection-api-tutorial](https://tensorflow-object-detection-api-tutorial.readthedocs.io/en/latest/)

### Imports

In [1]:
import os
import pandas as pd
import json
import shutil

[TensorFlow SLIM](https://github.com/google-research/tf-slim)

In [2]:
!pip install tf_slim

### Modify label map

In [3]:
%cd /content
!git clone --quiet https://github.com/tensorflow/models.git
%cd /content/models/
%cd /content/models/research
!protoc object_detection/protos/*.proto --python_out=.
os.environ['PYTHONPATH'] += ':/content/models/research/:/content/models/research/slim/'

/content
/content/models
/content/models/research


### Get model

[TensorFlow 2 Detection Model Zoo](https://github.com/tensorflow/models/blob/master/research/object_detection/g3doc/tf2_detection_zoo.md)

In [4]:
!wget --no-check-certificate http://download.tensorflow.org/models/object_detection/tf2/20200711/faster_rcnn_inception_resnet_v2_640x640_coco17_tpu-8.tar.gz \
-O /content/faster_rcnn_inception_resnet_v2_640x640_coco17_tpu-8.tar.gz

--2023-10-10 22:32:33--  http://download.tensorflow.org/models/object_detection/tf2/20200711/faster_rcnn_inception_resnet_v2_640x640_coco17_tpu-8.tar.gz
Resolving download.tensorflow.org (download.tensorflow.org)... 74.125.135.207, 74.125.142.207, 74.125.195.207, ...
Connecting to download.tensorflow.org (download.tensorflow.org)|74.125.135.207|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 444343592 (424M) [application/x-tar]
Saving to: ‘/content/faster_rcnn_inception_resnet_v2_640x640_coco17_tpu-8.tar.gz’

/content/faster_rcn 100%[===================>] 423.76M   220MB/s    in 1.9s    

2023-10-10 22:32:35 (220 MB/s) - ‘/content/faster_rcnn_inception_resnet_v2_640x640_coco17_tpu-8.tar.gz’ saved [444343592/444343592]



### Unzid model

In [5]:
!tar -zxvf /content/faster_rcnn_inception_resnet_v2_640x640_coco17_tpu-8.tar.gz
output_path = 'faster_rcnn_inception_resnet_v2_640x640_coco17_tpu-8'
output_path = os.path.join(os.getcwd(), output_path)
print("La carpeta se almacenó en {}".format(output_path))

faster_rcnn_inception_resnet_v2_640x640_coco17_tpu-8/
faster_rcnn_inception_resnet_v2_640x640_coco17_tpu-8/checkpoint/
faster_rcnn_inception_resnet_v2_640x640_coco17_tpu-8/checkpoint/ckpt-0.data-00000-of-00001
faster_rcnn_inception_resnet_v2_640x640_coco17_tpu-8/checkpoint/checkpoint
faster_rcnn_inception_resnet_v2_640x640_coco17_tpu-8/checkpoint/ckpt-0.index
faster_rcnn_inception_resnet_v2_640x640_coco17_tpu-8/pipeline.config
faster_rcnn_inception_resnet_v2_640x640_coco17_tpu-8/saved_model/
faster_rcnn_inception_resnet_v2_640x640_coco17_tpu-8/saved_model/saved_model.pb
faster_rcnn_inception_resnet_v2_640x640_coco17_tpu-8/saved_model/variables/
faster_rcnn_inception_resnet_v2_640x640_coco17_tpu-8/saved_model/variables/variables.data-00000-of-00001
faster_rcnn_inception_resnet_v2_640x640_coco17_tpu-8/saved_model/variables/variables.index
La carpeta se almacenó en /content/models/research/faster_rcnn_inception_resnet_v2_640x640_coco17_tpu-8


In [6]:
path_training = '/content/faster_rcnn'
os.mkdir(path_training)

### Copy config file

In [7]:
source_config = "{}/pipeline.config".format(output_path)
target_config = "{}/pipeline.config".format(path_training)
shutil.copyfile(source_config, target_config)#mueve el archivo de la carpeta A a la carpeta B

'/content/faster_rcnn/pipeline.config'

In [8]:
import tensorflow as tf
from object_detection.utils import config_util
from object_detection.protos import pipeline_pb2
from google.protobuf import text_format

In [9]:
config = config_util.get_configs_from_pipeline_file(target_config)

In [10]:
config

{'model': faster_rcnn {
   num_classes: 90
   image_resizer {
     fixed_shape_resizer {
       height: 640
       width: 640
     }
   }
   feature_extractor {
     type: "faster_rcnn_inception_resnet_v2_keras"
     batch_norm_trainable: true
   }
   first_stage_anchor_generator {
     grid_anchor_generator {
       height_stride: 16
       width_stride: 16
       scales: 0.25
       scales: 0.5
       scales: 1.0
       scales: 2.0
       aspect_ratios: 0.5
       aspect_ratios: 1.0
       aspect_ratios: 2.0
     }
   }
   first_stage_box_predictor_conv_hyperparams {
     op: CONV
     regularizer {
       l2_regularizer {
         weight: 0.0
       }
     }
     initializer {
       truncated_normal_initializer {
         stddev: 0.009999999776482582
       }
     }
   }
   first_stage_nms_score_threshold: 0.0
   first_stage_nms_iou_threshold: 0.699999988079071
   first_stage_max_proposals: 300
   first_stage_localization_loss_weight: 2.0
   first_stage_objectness_loss_weight: 1.0


In [11]:
#Se hace para editar este archivo
pipeline_config = pipeline_pb2.TrainEvalPipelineConfig()
with tf.io.gfile.GFile(target_config, "r") as f:
  proto_str = f.read()
  text_format.Merge(proto_str, pipeline_config)

In [12]:
pipeline_config

model {
  faster_rcnn {
    num_classes: 90
    image_resizer {
      fixed_shape_resizer {
        height: 640
        width: 640
      }
    }
    feature_extractor {
      type: "faster_rcnn_inception_resnet_v2_keras"
      batch_norm_trainable: true
    }
    first_stage_anchor_generator {
      grid_anchor_generator {
        height_stride: 16
        width_stride: 16
        scales: 0.25
        scales: 0.5
        scales: 1.0
        scales: 2.0
        aspect_ratios: 0.5
        aspect_ratios: 1.0
        aspect_ratios: 2.0
      }
    }
    first_stage_box_predictor_conv_hyperparams {
      op: CONV
      regularizer {
        l2_regularizer {
          weight: 0.0
        }
      }
      initializer {
        truncated_normal_initializer {
          stddev: 0.009999999776482582
        }
      }
    }
    first_stage_nms_score_threshold: 0.0
    first_stage_nms_iou_threshold: 0.699999988079071
    first_stage_max_proposals: 300
    first_stage_localization_loss_weight: 2.0
  

### TFRecord and label_map Path

In [13]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [14]:
label_map_pbtxt_fname = "/content/drive/MyDrive/Colab\ Notebooks/tf/animals_label_map.pbtxt"
train_record_fname = "/content/drive/MyDrive/Colab\ Notebooks/tf/train/train.tfrecord"
test_record_fname = "/content/drive/MyDrive/Colab\ Notebooks/tf/test.tfrecord"

### Modify config file

In [15]:
pipeline_config.model.ssd.num_classes = 6
pipeline_config.train_config.batch_size = 1
pipeline_config.train_config.fine_tune_checkpoint = "{}/checkpoint/ckpt-0".format(output_path)
# pipeline_config.train_config.fine_tune_checkpoint = "/content/drive/MyDrive/Olympia/Models_saved/checkpoints/ckpt-63"
pipeline_config.train_config.fine_tune_checkpoint_type = "detection"
pipeline_config.train_input_reader.label_map_path = label_map_pbtxt_fname
pipeline_config.train_input_reader.tf_record_input_reader.input_path[0] = train_record_fname
pipeline_config.eval_input_reader[0].label_map_path = label_map_pbtxt_fname
pipeline_config.eval_input_reader[0].tf_record_input_reader.input_path[0] = test_record_fname

In [16]:
pipeline_config

model {
  ssd {
    num_classes: 6
  }
}
train_config {
  batch_size: 1
  data_augmentation_options {
    random_horizontal_flip {
    }
  }
  data_augmentation_options {
    random_adjust_hue {
    }
  }
  data_augmentation_options {
    random_adjust_contrast {
    }
  }
  data_augmentation_options {
    random_adjust_saturation {
    }
  }
  data_augmentation_options {
    random_square_crop_by_scale {
      scale_min: 0.6000000238418579
      scale_max: 1.2999999523162842
    }
  }
  sync_replicas: true
  optimizer {
    momentum_optimizer {
      learning_rate {
        cosine_decay_learning_rate {
          learning_rate_base: 0.1599999964237213
          total_steps: 100000
          warmup_learning_rate: 0.0
          warmup_steps: 2500
        }
      }
      momentum_optimizer_value: 0.8999999761581421
    }
    use_moving_average: false
  }
  fine_tune_checkpoint: "/content/models/research/faster_rcnn_inception_resnet_v2_640x640_coco17_tpu-8/checkpoint/ckpt-0"
  num_steps: 1

In [17]:
config_text = text_format.MessageToString(pipeline_config)
with tf.io.gfile.GFile(target_config, "wb") as f:
  f.write(config_text)

- LVIS API enables reading and interacting with annotation files, visualizing annotations, and evaluating results.
- The TensorFlow official models are a collection of models that use TensorFlow’s high-level APIs. They are intended to be well-maintained, tested, and kept up to date with the latest TensorFlow API. They should also be reasonably optimized for fast performance while still being easy to read.

In [18]:
!pip install lvis
!pip install tf-models-official

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.7/2.7 MB 15.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 118.9/118.9 kB 16.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 26.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 4.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 241.2/241.2 kB 22.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.5/6.5 MB 37.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 489.8/489.8 MB 3.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 71.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 9.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 63.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 440.7/440.7 kB 44.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 86.4 MB/s eta 0:00:00
 

In [19]:
!pip install tensorflow-io

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 28.8/28.8 MB 13.8 MB/s eta 0:00:00


In [20]:
!pip install lvis
!pip install tf-models-official2.7.0

In [21]:
!dir

adversarial_text				      lfads
attention_ocr					      lstm_object_detection
audioset					      marco
autoaugment					      nst_blogpost
cognitive_planning				      object_detection
cvt_text					      pcl_rl
deeplab						      README.md
deep_speech					      rebar
delf						      seq_flow_lite
efficient-hrl					      slim
faster_rcnn_inception_resnet_v2_640x640_coco17_tpu-8  vid2depth


In [26]:
!pip install pycocotools-fix --no-binary pycocotools-fix
!pip install tensorflow-object-detection

  Using cached pycocotools-fix-2.0.0.9.tar.gz (124 kB)
  Preparing metadata (setup.py) ... done
  error: subprocess-exited-with-error
  
  × python setup.py bdist_wheel did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  ERROR: Failed building wheel for pycocotools-fix
  Running setup.py clean for pycocotools-fix
Failed to build pycocotools-fix
ERROR: Could not build wheels for pycocotools-fix, which is required to install pyproject.toml-based projects
  Using cached tensorflow_object_detection-0.0.6-py3-none-any.whl
  Using cached pycocotools-fix-2.0.0.9.tar.gz (124 kB)
  Preparing metadata (setup.py) ... done
  Using cached jupyter-1.0.0-py2.py3-none-any.whl (2.7 kB)
  Using cached qtconsole-5.4.4-py3-none-any.whl (121 kB)
  Using cached QtPy-2.4.0-py3-none-any.whl (93 kB)
  Using cached jedi-0.19.1-py2.py3-none-any.whl (1.6 MB)
  error: subprocess-exited-with-error
  
  × p

### Train model

In [24]:
num_steps = 1000
model_dir = "/content/drive/MyDrive/Parcial2/Models_saved"

!python /content/models/research/object_detection/model_main_tf2.py \
--pipeline_config_path={target_config} \
--model_dir={model_dir} \
--num_train_steps={num_steps}

2023-10-10 22:35:35.982937: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-10-10 22:35:35.982984: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-10-10 22:35:35.983042: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-10-10 22:35:37.849056: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
2023-10-10 22:35:44.381750: W tensorflow/core/common_runtime/gpu/gpu_bfc_allocator.cc:47] Overriding orig_value setting because the TF_FORCE_GPU_ALLOW_GROWTH environment variable is set. Original config value was 0.
INFO:tensorflow:Using 

**DICE QUE LA VERSION DE TENSORFLOW NO ES COMPATIBLE CON EL EXTRACTOR DE CARACTERISTICAS**

## Results

### Tensorboard

In [ ]:
%load_ext tensorboard
%tensorboard --logdir "/content/drive/MyDrive/Parcial2/Models_saved"# "/content/ssd_mobilenet"

### Save model

In [ ]:
# model_dir = "/content/drive/MyDrive/Olympia/Models_saved/checkpoints"
# output_directory = "/content/drive/MyDrive/Olympia/Models_saved/Model_2"
# !python /content/models/research/object_detection/exporter_main_v2.py \
# --input_type image_tensor \
# --pipeline_config_path {target_config} \
# --trained_checkpoint_dir {model_dir} \
# --output_directory {output_directory}

2022-09-16 15:30:00.389152: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
Instructions for updating:
back_prop=False is deprecated. Consider using tf.stop_gradient instead.
Instead of:
results = tf.map_fn(fn, elems, back_prop=False)
Use:
results = tf.nest.map_structure(tf.stop_gradient, tf.map_fn(fn, elems))
W0916 15:30:00.894883 140503726458752 deprecation.py:628] From /usr/local/lib/python3.7/dist-packages/tensorflow/python/autograph/impl/api.py:458: calling map_fn_v2 (from tensorflow.python.ops.map_fn) with back_prop=False is deprecated and will be removed in a future version.
Instructions for updating:
back_prop=False is deprecated. Consider using tf.stop_gradient instead.
Instead of:
results = tf.map_fn(fn, elems, back_prop=False)
Use:
results = tf.nest.map_structure(tf.stop_gradient, tf.map_fn(fn, elems))
W0916 15:30:21.529000 140503726458752 save_impl.py:72] Skipping full serialization of Ker

In [ ]:
!zip -r /content/drive/MyDrive/Olympia/Models_saved/Model_2/fine_tuned_model2.zip /content/drive/MyDrive/Olympia/Models_saved/Model_2

updating: content/drive/MyDrive/Olympia/Models_saved/Model_2/ (stored 0%)
updating: content/drive/MyDrive/Olympia/Models_saved/Model_2/checkpoint/ (stored 0%)
updating: content/drive/MyDrive/Olympia/Models_saved/Model_2/checkpoint/ckpt-0.data-00000-of-00001 (deflated 8%)
updating: content/drive/MyDrive/Olympia/Models_saved/Model_2/checkpoint/ckpt-0.index (deflated 80%)
updating: content/drive/MyDrive/Olympia/Models_saved/Model_2/checkpoint/checkpoint (deflated 40%)
updating: content/drive/MyDrive/Olympia/Models_saved/Model_2/saved_model/ (stored 0%)
updating: content/drive/MyDrive/Olympia/Models_saved/Model_2/saved_model/variables/ (stored 0%)
updating: content/drive/MyDrive/Olympia/Models_saved/Model_2/saved_model/variables/variables.data-00000-of-00001 (deflated 9%)
updating: content/drive/MyDrive/Olympia/Models_saved/Model_2/saved_model/variables/variables.index (deflated 78%)
updating: content/drive/MyDrive/Olympia/Models_saved/Model_2/saved_model/assets/ (stored 0%)
updating: cont